<a href="https://colab.research.google.com/github/komalkish/Amazon_vine_analysis/blob/master/Vine_review_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu 

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-02-12 21:46:04--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  --.-KB/s    in 0.08s   

2022-02-12 21:46:05 (11.4 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Pet_Products_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

### Create DataFrames to match tables

In [29]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame
petproducts_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
petproducts_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   28794885| REAKC26P07MDN|B00Q0K9604|     510387886|(8-Pack) EZwhelp ...|    Pet Products|          5|            0|          0|   N|                Y|A great purchase ...|Best belly bands ...| 2015-08-31|
|         US|   11488901|R3NU7OMZ4HQIEG|B00MBW5O9W|     912374672|Warren Eckstein's...|    Pet Products|          2|    

In [61]:
# Create the vine_table. DataFrame
vine_df = df.select("review_id", "star_rating","helpful_votes","total_votes","vine","verified_purchase")
vine_df.count()

2643619

In [63]:
total_votes_df = vine_df.filter(vine_df.total_votes>20)
total_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|                Y|
|R34X0ZGVB3YGYO|          5|           47|         47|   N|     

In [107]:
import pyspark.sql.functions as F

helpful_votes_df = total_votes_df.filter(total_votes_df['helpful_votes'] / total_votes_df['total_votes'] >0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|                Y|
|R34X0ZGVB3YGYO|          5|           47|         47|   N|     

In [109]:
#helpful_votes_df.show(10)
vine_yes_df = helpful_votes_df.filter(helpful_votes_df['vine'] == "Y")
vine_yes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3A71VR1JZD8WF|          2|           27|         30|   Y|                N|
|R16OMUJIGI18JZ|          5|           72|         72|   Y|                N|
|R3TS8ZP2FHQ9XR|          5|           39|         42|   Y|                N|
|R2MHP919VZN7DI|          5|           29|         30|   Y|                N|
|R1JUJLXQ2CMWKF|          4|           25|         26|   Y|                N|
|R2T7YE0IFI6N9L|          3|           24|         25|   Y|                N|
|R25FI3J7WWOYXM|          3|           55|         58|   Y|                N|
|R3P3U4EOWIUD50|          3|           27|         36|   Y|                N|
|R3C7MKX7MO6XUC|          5|          216|        219|   Y|                N|
|R1DFDDK1ZWKN3D|          2|           61|         65|   Y|     

In [110]:
vine_no_df = helpful_votes_df.filter(helpful_votes_df['vine'] == "N")
vine_no_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R21KC552Y6HL8X|          1|           27|         31|   N|                Y|
| RX9WC9FTIR1XR|          5|           25|         25|   N|                Y|
| RGDCOU1KBHMNG|          3|           29|         31|   N|                Y|
| RVTYWID2TPMMY|          2|           35|         42|   N|                Y|
|R2CMPZ5VESGRLY|          4|           27|         28|   N|                Y|
|R3VQPJZ54B55BA|          5|           62|         64|   N|                N|
|R24QM6D7FEDZ5M|          2|           36|         43|   N|                Y|
|  R39GSNW76GYF|          1|           20|         23|   N|                Y|
| RPJLR6MFDPXXE|          5|           35|         36|   N|                Y|
|R34X0ZGVB3YGYO|          5|           47|         47|   N|     

In [132]:
total_rating = helpful_votes_df.select(helpful_votes_df["star_rating"]).count()
total_rating

35730

In [122]:
yes_star_rating = vine_yes_df.filter(vine_yes_df["star_rating"] >= 5).count()
yes_star_rating

63

In [123]:
no_star_rating = vine_no_df.filter(vine_no_df["star_rating"] >=5).count()
no_star_rating


19437

In [139]:
paid_rating_percentage = yes_star_rating/total_rating 
paid_rating_percentage

0.00017632241813602016

In [137]:
unpaid_rating_percentage = no_star_rating/total_rating
unpaid_rating_percentage

0.5439966414777498